In [ ]:
import tensorflow as tf
from data import DataGenerator
from model import espcn, psnr
from training import train

r = 3 # Upscale Factor
batch_size = 10
epochs = 100

# Compile model
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
# Peak Signal-to-Noise Ratio
def PSNR(y_true, y_pred):
    max_pixel = 1.0
    return tf.image.psnr(y_true, y_pred, max_val=max_pixel)
model = espcn(r)
model.compile(optimizer=opt, loss='mse', metrics=[PSNR])
model.summary()

In [ ]:
# Fit model.
train(model, r, batch_size, epochs)
# Weights are saved to 'model/weights/r[r]bs[batch_size]epochs[epochs]weights.h5'